# Ensamble Technique Assignment 5

In [1]:
import seaborn as sns
df = sns.load_dataset('iris')
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [2]:
df.describe()

,sepal_length,sepal_width,petal_length,petal_width
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333
std,0.828066,0.435866,1.765298,0.762238
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [4]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['species'] = encoder.fit_transform(df['species'])

In [5]:
x = df.drop('species', axis=1)
y = df['species']

In [6]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.30,random_state=42)

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

In [8]:
numerical_cols = ['sepal_length','sepal_width','petal_length','petal_width']
category_cols = []

In [9]:
numaric_pipeline = Pipeline(
     steps=[
         ('imputer',SimpleImputer(strategy='median')),
         ('scaling',StandardScaler())
     ]
)

category_pipeline = Pipeline(
        steps = [
         ('imputer',SimpleImputer(strategy='most_frequent')),
         ('onehotencoder',OneHotEncoder())
        ]
)

In [10]:
preprocesser = ColumnTransformer([
         ('numaric_pipeline',numaric_pipeline,numerical_cols),
         ('category_pipeline',category_pipeline,category_cols)
])

In [11]:
X_train = preprocesser.fit_transform(x_train)
X_test = preprocesser.transform(x_test)

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [13]:
models = {
      'Random Forest': RandomForestClassifier(),
      'Logistic Regression':LogisticRegression(),
      'Decision Tree': DecisionTreeClassifier(),
}

In [16]:
def evatute_cols(X_train,y_train,X_test,y_test,models):
    
    report = {}
    for i in range(len(models)):
        model = list(models.values())[i]
        
        model.fit(X_train,y_train)
        
        
        y_pred = model.predict(X_test)
        
        model_score = accuracy_score(y_pred,y_test)
        
        report[list(models.keys())[i]] = model_score
        
        
    return report
  

In [17]:
evatute_cols(X_train,y_train,X_test,y_test,models)

{'Random Forest': 1.0, 'Logistic Regression': 1.0, 'Decision Tree': 1.0}

In [18]:
classifier = RandomForestClassifier()

In [24]:
parameters = {'max_depth':[3,5,10,None],
              'n_estimators':[100,200,300],
               'criterion':['gini','entropy']
              }

In [25]:
from sklearn.model_selection import RandomizedSearchCV
import warnings
warnings.filterwarnings('ignore')

In [26]:
clf = RandomizedSearchCV(classifier,param_distributions=parameters,cv=5,scoring='accuracy',verbose=3)

In [27]:
clf.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=entropy, max_depth=None, n_estimators=200;, score=0.952 total time=   0.3s
[CV 2/5] END criterion=entropy, max_depth=None, n_estimators=200;, score=0.905 total time=   0.4s
[CV 3/5] END criterion=entropy, max_depth=None, n_estimators=200;, score=0.905 total time=   0.3s
[CV 4/5] END criterion=entropy, max_depth=None, n_estimators=200;, score=1.000 total time=   0.3s
[CV 5/5] END criterion=entropy, max_depth=None, n_estimators=200;, score=0.952 total time=   0.3s
[CV 1/5] END criterion=gini, max_depth=5, n_estimators=300;, score=0.952 total time=   0.4s
[CV 2/5] END criterion=gini, max_depth=5, n_estimators=300;, score=0.905 total time=   0.4s
[CV 3/5] END criterion=gini, max_depth=5, n_estimators=300;, score=0.905 total time=   0.4s
[CV 4/5] END criterion=gini, max_depth=5, n_estimators=300;, score=1.000 total time=   0.4s
[CV 5/5] END criterion=gini, max_depth=5, n_estimators=300;, score=0.952 total ti

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, 5, 10, None],
                                        'n_estimators': [100, 200, 300]},
                   scoring='accuracy', verbose=3)

In [28]:
clf.best_params_

{'n_estimators': 200, 'max_depth': None, 'criterion': 'entropy'}

In [29]:
best_model = clf.best_estimator_
best_model

RandomForestClassifier(criterion='entropy', n_estimators=200)

In [30]:
classifier = RandomForestClassifier(n_estimators=200,max_depth=None,criterion='entropy')
classifier.fit(X_train,y_train)

RandomForestClassifier(criterion='entropy', n_estimators=200)

In [31]:
y_pred = classifier.predict(X_test)

In [32]:
from sklearn.metrics import confusion_matrix,classification_report
conf_matrix = confusion_matrix(y_pred,y_test)
clf_report = classification_report(y_pred,y_test)
score = accuracy_score(y_pred,y_test)
print(conf_matrix)
print(clf_report)
print(score)

[[19  0  0]
 [ 0 13  0]
 [ 0  0 13]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       1.00      1.00      1.00        13
           2       1.00      1.00      1.00        13

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45

1.0
